# LBM Step 4: Collate Planes

In [ ]:
import sys
from pathlib import Path
import os
import numpy as np
import zarr
import napari

import logging
import matplotlib.pyplot as plt

# Importing this WONT WORK withotu adding to the path
sys.path.append('../..')  # TODO: Take this out when we upload to pypi
import lbm_caiman_python as lcp

import mesmerize_core as mc

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

logging.basicConfig()

os.environ["CONDA_PREFIX_1"] = ""
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

debug = True

logger = logging.getLogger("caiman")
logger.setLevel(logging.WARNING)
handler = logging.StreamHandler()
log_format = logging.Formatter("%(relativeCreated)12d [%(filename)s:%(funcName)10s():%(lineno)s] [%(process)d] %(message)s")
handler.setFormatter(log_format)
logger.addHandler(handler)

# set env variables
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"

if debug:
   logging.getLogger("caiman").setLevel(logging.WARNING)

In [4]:
raw_data_path = Path().home() / "caiman_data"
movie_path = raw_data_path / 'animal_01' / "session_01" / 'save_gui.zarr'

In [5]:
batch_path = raw_data_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(raw_data_path))

PosixPath('/home/mbo/caiman_data')

In [7]:
# create a new batch
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 9781250d-22cb-4ebc-9d...,2024-09-06T00:19:51,2024-09-06T00:33:05,764.98 sec,None,9781250d-22cb-4ebc-9de1-ad97e101ddee
1,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 5ffcad60-5506-4243-93...,2024-09-06T00:19:51,2024-09-06T00:35:31,142.87 sec,None,5ffcad60-5506-4243-936c-f8473ff4ab50
2,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 0a463c4f-985d-47a4-9e...,2024-09-06T00:19:51,2024-09-06T00:48:21,766.17 sec,None,0a463c4f-985d-47a4-9e95-cf776d98a1a3
3,mcorr,save_gui,animal_01/session_01/save_gui.zarr,"{'main': {'var_name_hdf5': 'plane_2', 'max_shi...",{'mean-projection-path': 8cb260f3-ced8-4433-b9...,2024-09-06T00:19:51,2024-09-06T00:50:47,142.94 sec,None,8cb260f3-ced8-4433-b9bf-fd6116b20964
4,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 3db5aac3-b8ab-449c-9b...,2024-09-06T02:52:12,2024-09-06T02:54:12,100.68 sec,None,3db5aac3-b8ab-449c-9b13-be7e6eaa19a7
5,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 796af0f5-678c-4a73-99...,2024-09-06T02:55:26,2024-09-06T02:57:21,100.82 sec,None,796af0f5-678c-4a73-9991-d8d462e9d66a
6,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': e58bb527-8bd1-4652-82...,2024-09-06T02:55:26,2024-09-06T02:59:07,102.67 sec,None,e58bb527-8bd1-4652-82dd-bca82327a018
7,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 4ae126bc-4bb1-4277-94...,2024-09-06T02:55:26,2024-09-06T03:01:48,158.05 sec,None,4ae126bc-4bb1-4277-946d-dbe472fde572
8,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': cf20ff30-e1cb-45ff-91...,2024-09-06T02:55:26,2024-09-06T03:04:37,166.19 sec,None,cf20ff30-e1cb-45ff-912a-c99b4a5939a3
9,cnmf,save_gui,8cb260f3-ced8-4433-b9bf-fd6116b20964/8cb260f3-...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 72001e65-2c76-4271-bb...,2024-09-06T02:55:26,2024-09-06T03:06:32,111.71 sec,None,72001e65-2c76-4271-bbd6-72b95ec8673e


In [11]:
traces = df.iloc[-1].cnmf.get_temporal()
traces.shape

(4629, 1730)

In [12]:
traces[0,0]

-1346.7363421255025